In [1]:
import pandas as pd

In [2]:
db = pd.read_excel('столы и стулья.xlsx')

In [3]:
db

,фраза,группа 1,группа 2
0,Подскажите пожалуйста информацию о столе,стол,продукт
1,стол,стол,продукт
2,какие у вас есть столы?,стол,продукт
3,мне очень нравятся ваши столы,стол,цена
4,у вас очень дорогие столы,стол,цена
5,какие у вас большие цены на столы,стол,цена
6,Подскажите пожалуйста информацию о стуле,стулья,продукт
7,стул,стулья,продукт
8,какие у вас есть стулья?,стулья,продукт
9,мне очень нравятся ваши стулья,стулья,цена


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
import re
regex = re.compile("[А-Яа-я]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
noise = stopwords.words('russian')
noise = noise+['это', 'тыс', 'млн', 'млрд', 'также']
mystoplemmas = ['который','прошлый','сей', 'свой', 'наш','очень', 'мочь','этот', 'такой', ' ', '\n']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\volko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def dataprep(inf):
    dp = [words_only(x.lower()) for x in inf]
    dp = [x.replace(' ',',').split(',') for x in dp]
    long_words = []
    for j in tqdm(dp):
        long_words.append([x for x in j if len(x)>1])
    latterm_wo_stopw = []
    for k in tqdm(long_words):
        latterm_wo_stopw.append(' '.join([x for x in k if x not in noise]))
    return latterm_wo_stopw

In [8]:
from rnnmorph.predictor import RNNMorphPredictor
predictor=RNNMorphPredictor(language='ru')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
def lem(a):
    res = []
    for i in tqdm(a):
        i = i.split(' ')
        forms = predictor.predict(i)
        res.append([word.normal_form for word in forms])
    return res

In [10]:
def lem_wo_stop(a):
    latterm = []
    for i in tqdm(a):
        latterm.append(' '.join([x for x in i if x not in mystoplemmas]))
    return latterm

In [11]:
from tqdm import tqdm

In [12]:
data = dataprep(db['фраза'])

100%|██████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<?, ?it/s]


In [13]:
data = lem(data)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 18.32it/s]


In [14]:
data = lem_wo_stop(data)

100%|████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 6014.78it/s]


In [15]:
tfidf = TfidfVectorizer(ngram_range=(1,2))

In [16]:
tfidf_data = tfidf.fit_transform(data)

In [17]:
target1 = [1 if x == 'стол' else 0 for x in db['группа 1']]

In [18]:
from  sklearn.linear_model import LogisticRegression

In [19]:
model1=LogisticRegression()

In [20]:
model1.fit(tfidf_data, target1)

C:\Users\volko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [21]:
tfidf.get_feature_names()

['большой',
 'большой цена',
 'ваш',
 'ваш стол',
 'ваш стул',
 'дорогой',
 'дорогой стол',
 'дорогой стул',
 'информация',
 'информация стол',
 'информация стул',
 'какой',
 'какой большой',
 'какой стол',
 'какой стул',
 'нравиться',
 'нравиться ваш',
 'подсказать',
 'подсказать пожалуйста',
 'пожалуйста',
 'пожалуйста информация',
 'стол',
 'стул',
 'цена',
 'цена стол',
 'цена стул']

In [22]:
z1 = ['Что я знаю о стуле']
z2 = ['Расскажите мне об этом стуле']
z3 = ['Какой красивый стол']
z4 = ['Здесь так много столов']

In [23]:
import numpy as np

In [24]:
prediction=model1.predict(tfidf.transform((lem_wo_stop(lem(dataprep(z1))))))
print(z1)
print('cтол' if prediction==1 else 'стул')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


['Что я знаю о стуле']
стул


In [25]:
prediction=model1.predict(tfidf.transform((lem_wo_stop(lem(dataprep(z2))))))
print(z2)
print('cтол' if prediction==1 else 'стул')

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1003.90it/s]


['Расскажите мне об этом стуле']
стул


In [26]:
prediction=model1.predict(tfidf.transform((lem_wo_stop(lem(dataprep(z3))))))
print(z3)
print('cтол' if prediction==1 else 'стул')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


['Какой красивый стол']
cтол


In [27]:
prediction=model1.predict(tfidf.transform((lem_wo_stop(lem(dataprep(z4))))))
print(z4)
print('cтол' if prediction==1 else 'стул')

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 999.83it/s]


['Здесь так много столов']
cтол


In [28]:
target2 = [1 if x == 'продукт' else 0 for x in db['группа 2']]

In [29]:
model2=LogisticRegression()
model2.fit(tfidf_data, target2)

C:\Users\volko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [30]:
prediction=model2.predict(tfidf.transform((lem_wo_stop(lem(dataprep(z1))))))
print(z1)
print('продукт' if prediction==1 else 'цена')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


['Что я знаю о стуле']
продукт


In [31]:
z5=['у вас стулья дороже']

In [32]:
prediction=model2.predict(tfidf.transform((lem_wo_stop(lem(dataprep(z5))))))
print(z5)
print('продукт' if prediction==1 else 'цена')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


['у вас стулья дороже']
цена


In [ ]:
остается проблема с синонимами- рещается word2vec или объемом данных (больше комбинаций запросов на тему)

In [ ]:
и про стоп слова- слово очень было в данных, а теперь оно никак не фигурирует в модели (см.ячейку 21)